### 10 Ollama 설치 후 Modelfile 설정하기

- 허깅페이스(HuggingFace) 오픈 모델 페이지를 열고 **Files and versions** 탭에서 오픈모델을 다운로드 (.gguf 확장자)
- 모델 설치한 경로에 ModelFile 생성. 해당 모델에서 공개한 양식을 복사해서 붙여넣기

```
FROM EEVE-Korean-Instruct-10.8B-v1.0-Q4_0.gguf

TEMPLATE """{{- if .System }}
<s>{{ .System }}</s>
{{- end }}
<s>Human:
{{ .Prompt }}</s>
<s>Assistant:
"""

SYSTEM """A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions."""

PARAMETER stop <s>
PARAMETER stop </s>
```

- 템플릿 안에 \<s\>, \</s\>로 묶은 부분은 **스페셜 토큰**, 출력되는 프롬프트의 시작과 끝을 지정

### 11 Ollama 모델 생성하고 ChatOllama 활용하기

- Ollama에 업로드된 기성 모델을 다운로드해서 모델을 만들고 실행

**Ollama 모델 생성하기**

- `ollama list`로 현재 설치된 모델 확인
- `ollama pull <name-of-model>` 명령어로 Ollama에서 제공하는 모델 다운로드
- `ollama create <model-name> -f <path-to-modelfile>` 명령어로 Modelfile로부터 커스텀 모델 생성

**Ollama 모델과 LangChain 활용하기**

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging
logging.langsmith("CH04-Models")

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_teddynote.messages import stream_response

# Ollama 모델을 불러옵니다.
llm = ChatOllama(model="EEVE-Korean-Instruct-10.8B-v1.0-Q4_0.gguf:latest")

# 프롬프트
prompt = ChatPromptTemplate.from_template("{topic} 에 대하여 간략히 설명해 줘.")

# 체인 생성
chain = prompt | llm | StrOutputParser()

# 간결성을 위해 응답은 터미널에 출력됩니다.
answer = chain.stream({"topic": "deep learning"})

# 스트리밍 출력
stream_response(answer)

**JSON 형식으로 출력하기**

- `format` 플래그는 모델이 JSON 형식으로 응답을 생성하도록 강제

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="gemma:7b",  # 사용할 언어 모델을 지정합니다.
    format="json",  # 입출력 형식을 JSON으로 설정합니다.
    temperature=0,
)

- JSON 형식의 답변을 받기 위해서는 `"response in JSON format."` 이 프롬프트에 포함되어야 한다

In [ ]:
# JSON 형식의 답변을 요구하는 프롬프트 작성
prompt = "유럽 여행지 10곳을 알려주세요. key: `places`. response in JSON format."

# 체인 호출
response = llm.invoke(prompt)
print(response.content)  # 생성된 응답을 출력합니다.

**멀티모달(Multimodal) 지원**

- Ollama는 [bakllava](https://ollama.ai/library/bakllava)와 [llava](https://ollama.ai/library/llava)와 같은 멀티모달 LLM을 지원
- `tags`를 사용하여 [Llava](https://ollama.ai/library/llava/tags)와 같은 모델의 전체 버전 세트를 탐색 가능
- `ollama pull llava:7b` 혹은 `ollama pull bakllava` 명령어를 통해 멀티모달 LLM을 다운로드
- **참고**: 멀티모달을 지원하는 최신 버전을 사용하려면 Ollama를 업데이트

- 이미지를 인식하는 멀티모달을 구현하려면 먼저 이미지를 base64로 인코딩된 문자열로 변환하는 코드 필요
- PIL 이미지를 Base64 인코딩된 문자열로 변환하고 이를 HTML에 포함하여 이미지를 표시하는 함수를 제공
- `convert_to_base64` 함수:
  - PIL 이미지를 입력으로 받는다.
  - 이미지를 JPEG 형식으로 BytesIO 버퍼에 저장
  - 버퍼의 값을 Base64로 인코딩하고 문자열로 반환
- `plt_img_base64` 함수:
  - Base64 인코딩된 문자열을 입력
  - Base64 문자열을 소스로 사용하는 HTML `<img>` 태그를 생성
  - HTML을 렌더링하여 이미지를 표시
- 사용 예시:
  - 지정된 파일 경로에서 PIL 이미지를 열어 `pil_image`에 저장
  - `convert_to_base64` 함수를 사용하여 `pil_image`를 Base64 인코딩된 문자열로 변환
  - `plt_img_base64` 함수를 사용하여 Base64 인코딩된 문자열을 이미지로 표시.


In [ ]:
import base64
from io import BytesIO

from IPython.display import HTML, display
from PIL import Image
from langchain_core.messages import HumanMessage


def convert_to_base64(pil_image):
    """
    PIL 이미지를 Base64로 인코딩된 문자열로 변환합니다.

    :param pil_image: PIL 이미지
    :return: 크기 조정된 Base64 문자열
    """

    buffered = BytesIO()
    pil_image.save(buffered, format="JPEG")  # 필요한 경우 형식을 변경할 수 있습니다.
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str


def plt_img_base64(img_base64):
    """
    Base64로 인코딩된 문자열을 이미지로 표시합니다.

    :param img_base64:  Base64 문자열
    """
    # Base64 문자열을 소스로 사용하여 HTML img 태그 생성
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'
    # HTML을 렌더링하여 이미지 표시
    display(HTML(image_html))


def prompt_func(data):  # 프롬프트 함수를 정의합니다.
    text = data["text"]  # 데이터에서 텍스트를 가져옵니다.
    image = data["image"]  # 데이터에서 이미지를 가져옵니다.

    image_part = {  # 이미지 부분을 정의합니다.
        "type": "image_url",  # 이미지 URL 타입을 지정합니다.
        "image_url": f"data:image/jpeg;base64,{image}",  # 이미지 URL을 생성합니다.
    }

    content_parts = []  # 콘텐츠 부분을 저장할 리스트를 초기화합니다.

    text_part = {"type": "text", "text": text}  # 텍스트 부분을 정의합니다.

    content_parts.append(image_part)  # 이미지 부분을 콘텐츠 부분에 추가합니다.
    content_parts.append(text_part)  # 텍스트 부분을 콘텐츠 부분에 추가합니다.

    return [HumanMessage(content=content_parts)]  # HumanMessage 객체를 반환합니다.


file_path = "images/jeju-beach.jpg"
pil_image = Image.open(file_path)

image_b64 = convert_to_base64(pil_image)

plt_img_base64(image_b64)

- `ChatOllama` 언어 모델을 사용하여 이미지와 텍스트 기반 질의에 대한 답변을 생성하는 체인을 구현.
- `prompt_func` 함수는 이미지와 텍스트 데이터를 입력으로 받아 `HumanMessage` 형식으로 변환.
  - 이미지 데이터는 Base64 인코딩된 JPEG 형식으로 전달
  - 텍스트 데이터는 일반 텍스트로 전달
- `StrOutputParser`를 사용하여 언어 모델의 출력을 문자열로 파싱.
- `prompt_func`, `llm`, `StrOutputParser`를 파이프라인으로 연결하여 `chain`을 생성
- `chain.invoke` 메서드를 호출하여 이미지와 텍스트 질의를 전달하고 답변을 생성
- 생성된 답변을 출력

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage

# ChatOllama 멀티모달 언어 모델을 불러옵니다.
llm = ChatOllama(model="llava:7b", temperature=0)

# 프롬프트 함수, 언어 모델, 출력 파서를 연결하여 체인을 생성합니다.
chain = prompt_func | llm | StrOutputParser()

query_chain = chain.invoke(  # 체인을 호출하여 쿼리를 실행합니다.
    # 텍스트와 이미지를 전달합니다.
    {"text": "Describe a picture in bullet points", "image": image_b64}
)

print(query_chain)  # 쿼리 결과를 출력합니다.
